In [1]:
import cv2
import numpy as np
from PIL import Image, ImageFilter

In [2]:
# Read the video from specified path
cam = cv2.VideoCapture("./new/encryptedvideo.avi")
# frames = cam.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cam.get(cv2.CAP_PROP_FPS)
# seconds = round(frames / fps)
print(fps)

30.0


In [3]:
image='./SquareDetectionResults/'
ext='.bmp'

In [4]:
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm

In [5]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],color

In [6]:
def LogisticDecryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]

    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    
    L_x = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L_x *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1

    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    
    C = round((L_x*L_y*10**4) % 256)
    I_prev = C
    I_prev_r = C
    I_prev_g = C
    I_prev_b = C
    I = C
    I_r = C
    I_g = C
    I_b = C
    x_prev = 4*(S_x)*(1-S_x)
    y_prev = 4*(L_x)*(1-S_y)
    x = x_prev
    y = y_prev
    imageMatrix, dimension, color = getImageMatrix(imageName)

    henonDecryptedImage = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            while x <0.8 and x > 0.2 :
                x = 4*x*(1-x)
            while y <0.8 and y > 0.2 :
                y = 4*y*(1-y)
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                I_r = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_r + key_list[7]) % N) ^ imageMatrix[i][j][0]) + N-key_list[6])%N
                I_g = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_g + key_list[7]) % N) ^ imageMatrix[i][j][1]) + N-key_list[6])%N
                I_b = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_b + key_list[7]) % N) ^ imageMatrix[i][j][2]) + N-key_list[6])%N
                I_prev_r = imageMatrix[i][j][0]
                I_prev_g = imageMatrix[i][j][1]
                I_prev_b = imageMatrix[i][j][2]
                row.append((I_r,I_g,I_b))
                x = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1  
            else:
                I = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev+key_list[7]) % N) ^ imageMatrix[i][j]) + N-key_list[6])%N
                I_prev = imageMatrix[i][j]
                row.append(I)
                x = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        henonDecryptedImage.append(row)
    if color:
        im = Image.new("RGB", (dimension, dimension))
    else: 
        im = Image.new("L", (dimension, dimension)) # L is for Black and white pixels
    pix = im.load()
    for x in range(dimension):
        for y in range(dimension):
            pix[x, y] = henonDecryptedImage[x][y]
    im.save(image+"decryp"+imageName[28:-4]+ext, "BMP")

In [7]:
key = "abcdefghijklm"

In [8]:
folder_dir = "./SquareDetectionResults/encrypted_frames"
cropped_images = []
for images in os.listdir(folder_dir):
    # Open image and make into Numpy array
    im = Image.open('./SquareDetectionResults/ppl1.png').convert('RGB')
    na = np.array(im)
    orig = na.copy()    # Save original
    # Median filter to remove outliers
    im = im.filter(ImageFilter.MedianFilter(3))
    # Find X,Y coordinates of all yellow pixels
    yellowY, yellowX = np.where(np.all(na==[0, 0, 255],axis=2))
    print(yellowX)
    #yellowY2, yellowX2= np.where(np.all(na==[0, 0, 255],axis=-2))
    zipped = np.column_stack((yellowX,yellowY))
    print("xipp",zipped)
    uniqueY = list(yellowY)
    uniqueX= list(set(yellowX))
    uniquesetX={uniqueX[0]}
    uniquesetY={uniqueY[0]}
    for x in range(0,len(uniqueX)-1):
      if(uniqueX[x+1]-uniqueX[x]>=2):
        uniquesetX.add(uniqueX[x])
        uniquesetX.add(uniqueX[x+1])
    uniquesetX.add(uniqueX[len(uniqueX)-1])
    uniquesetX=sorted(uniquesetX)
    uniquesetY=list()
    for a in uniquesetX:
      for x,y in zipped:
        if(a==x):
          uniquesetY.append(y)
    uniqueset2Y=list()
    uniqueset2Y.append(uniquesetY[0])
    for x in range(0,len(uniquesetY)-1):
      if(uniquesetY[x+1]-uniquesetY[x]!=1):
        uniqueset2Y.append(uniquesetY[x])
        uniqueset2Y.append(uniquesetY[x+1])
    uniqueset2Y.append(uniquesetY[len(uniquesetY)-1])
    print("uniqesetY",uniqueset2Y)
    uniqueset3Y=list()
    for x in range(0,len(uniqueset2Y),4):
      uniqueset3Y.append(uniqueset2Y[x])
      uniqueset3Y.append(uniqueset2Y[x+1])
    print("tellmefinal")
    print(uniqueset3Y)
    img = cv2.imread('./SquareDetectionResults/ppl1.png')
    faces=[]
    for i in range(0,len(uniquesetX)-1,2):
        a=uniqueset3Y[i]
        b=uniqueset3Y[i+1]
        c=uniquesetX[i]
        d=uniquesetX[i+1]
        face=img[a+1:b,c+1:d]
        filename=image+ext
        cv2.imwrite(filename, face)
        LogisticDecryption(filename, key)
        filename=image+"decryp"+ext
        DecryptResult=cv2.imread(filename)
        img[a+1:b,c+1:d]=DecryptResult
        cv2.imwrite('./SquareDetectionResults/decrypted_frames/DecryptionResult'+images, img)

[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
uniqesetY [161, 353, 161, 353, 233, 409, 233, 409, 184, 373, 184, 373]
tellmefinal
[161, 353, 233, 409, 184, 373]
[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
uniqesetY [161, 353, 161, 353, 233, 409, 233, 409, 184, 373, 184, 373]
tellmefinal
[161, 353, 233, 409, 184, 373]
[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
uniqesetY [161, 353, 161, 353, 233, 409, 233, 409, 184, 373, 184, 373]
tellmefinal
[161, 353, 233, 409, 184, 373]
[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
uniqesetY [161, 353, 161, 353, 233, 409, 233, 409, 184, 373, 184, 373]
tellmefinal
[161, 353, 233, 409, 184, 373]
[ 320  321  322 ... 1016 1017 1018]
xipp [[ 

[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
uniqesetY [161, 353, 161, 353, 233, 409, 233, 409, 184, 373, 184, 373]
tellmefinal
[161, 353, 233, 409, 184, 373]
[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
uniqesetY [161, 353, 161, 353, 233, 409, 233, 409, 184, 373, 184, 373]
tellmefinal
[161, 353, 233, 409, 184, 373]
[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
uniqesetY [161, 353, 161, 353, 233, 409, 233, 409, 184, 373, 184, 373]
tellmefinal
[161, 353, 233, 409, 184, 373]
[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
uniqesetY [161, 353, 161, 353, 233, 409, 233, 409, 184, 373, 184, 373]
tellmefinal
[161, 353, 233, 409, 184, 373]
[ 320  321  322 ... 1016 1017 1018]
xipp [[ 